In [0]:
## Place this cell in any team notebook that needs access to the team cloud storage.


# The following blob storage is accessible to team members only (read and write)
# access key is valid til TTL
# after that you will need to create a new SAS key and authenticate access again via DataBrick command line
blob_container  = "final-project-summer24-team3"       # The name of your container created in https://portal.azure.com
storage_account = "summer2024team3"  # The name of your Storage account created in https://portal.azure.com
secret_scope    = "summer24_team_3_2_scope"           # The name of the scope created in your local computer using the Databricks CLI
secret_key      = "final-project-summer24-team3"             # The name of the secret key created in your local computer using the Databricks CLI
team_blob_url   = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"  #points to the root of your team storage bucket


# the 261 course blob storage is mounted here.
mids261_mount_path      = "/mnt/mids-w261"

# SAS Token: Grant the team limited access to Azure Storage resources
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
from pyspark.sql.functions import col, concat_ws, lpad, expr, unix_timestamp, from_unixtime, when, count, lag, greatest, row_number,lit
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, DoubleType, FloatType, StructField, StructType
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time as t
from pyspark.ml.feature import Bucketizer, SQLTransformer, OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from itertools import chain
from pyspark.mllib.evaluation import MulticlassMetrics
from xgboost.spark import SparkXGBClassifier, SparkXGBRegressor

## Load All DataFrames

In [0]:
display(dbutils.fs.ls(f"{team_blob_url}/TP/"))

path,name,size,modificationTime
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_SUCCESS,_SUCCESS,0,1720561571000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_committed_1002670972388615845,_committed_1002670972388615845,625,1720560531000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_committed_5669257934384103852,_committed_5669257934384103852,221,1720561571000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_committed_6618439955609603938,_committed_6618439955609603938,419,1720561337000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_committed_9167039456723159873,_committed_9167039456723159873,318,1720559468000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_committed_vacuum825530481471543349,_committed_vacuum825530481471543349,95,1720561338000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_started_1002670972388615845,_started_1002670972388615845,0,1720560531000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_started_5669257934384103852,_started_5669257934384103852,0,1720561570000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/_started_6618439955609603938,_started_6618439955609603938,0,1720561337000
wasbs://final-project-summer24-team3@summer2024team3.blob.core.windows.net/TP/df_1y_cleaned_transformed/,df_1y_cleaned_transformed/,0,1722046865000


In [0]:
xgboost1011 = spark.read.parquet(f'{team_blob_url}/TP/xgboost_fbeta_recall')
xgboost1066 = spark.read.parquet(f'{team_blob_url}/TP/xgboost_precision')
log_base = spark.read.parquet(f'{team_blob_url}/TP/logbaseline_probabilities_2018_2019/')
log_variant = spark.read.parquet(f'{team_blob_url}/TP/logbaseline_0.03_probabilities_2018_2019/')
nnval2 = spark.read.parquet(f'{team_blob_url}/TP/nn_preds_val2')
nnval5 = spark.read.parquet(f'{team_blob_url}/TP/nn_preds_val5')
nntest2 = spark.read.parquet(f'{team_blob_url}/TP/nn_preds_test2')
nntest5 = spark.read.parquet(f'{team_blob_url}/TP/nn_preds_test5')

## Create Train Data

In [0]:
nnval2 = nnval2.withColumnRenamed("probability", "nnval2_prob")
merged_nn = nnval2.join(nnval5, "index", "left") \
    .select("index", "nnval2_prob", F.col("probability").alias("nnval5_prob"))
merged_log_base = merged_nn.join(log_base, "index", "left") \
    .select("index", "nnval2_prob", "nnval5_prob", 
            F.col("DEP_DEL15").alias("target"), F.col("probability_1").alias("log_base_prob"), "YEAR")
merged_log_all = merged_log_base.join(log_variant, "index", "left") \
    .select("index", "nnval2_prob", "nnval5_prob", "log_base_prob",
            F.col("probability_1").alias("log_variant_prob"), "target")
merged_boost11 = merged_log_all.join(xgboost1011, "index", "left")\
    .select("index", "nnval2_prob", "nnval5_prob", "log_base_prob", "log_variant_prob",
            F.col("probability_1").alias("xgboost1011_prob"), "target")
val_df = merged_boost11.join(xgboost1066, "index", "left")\
    .select("index", "nnval2_prob", "nnval5_prob", "log_base_prob", "log_variant_prob", "xgboost1011_prob",
            F.col("probability_1").alias("xgboost1066_prob"), "target")

In [0]:
# CheckSum for the train dataset

# Count total number of rows
total_rows = val_df.count()
print(f"Total number of rows: {total_rows}")

# Check for null data in any column
null_counts = val_df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in val_df.columns])
null_counts.show()

# Count rows with any null value
rows_with_nulls = val_df.filter(F.concat_ws("", *val_df.columns).isNull()).count()
print(f"Number of rows with null values: {rows_with_nulls}")

Total number of rows: 3606364
+-----+-----------+-----------+-------------+----------------+----------------+----------------+------+
|index|nnval2_prob|nnval5_prob|log_base_prob|log_variant_prob|xgboost1011_prob|xgboost1066_prob|target|
+-----+-----------+-----------+-------------+----------------+----------------+----------------+------+
|    0|          0|          0|            0|               0|               0|               0|     0|
+-----+-----------+-----------+-------------+----------------+----------------+----------------+------+

Number of rows with null values: 0


### Transform Train Data

In [0]:
feature_columns = ["nnval2_prob", "nnval5_prob", "log_base_prob", "log_variant_prob", "xgboost1011_prob", "xgboost1066_prob"]
assembler = VectorAssembler(
    inputCols=[x for x in feature_columns],
    outputCol="features",
    handleInvalid = "skip"
)

pipeline = Pipeline(stages=[assembler])
val_trf = pipeline \
    .fit(val_df) \
    .transform(val_df) \
    .select("index", "features", "target")

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

## Helper Functions

In [0]:
def block_splits(n_splits, df):
    """
    Split DataFrame into blocks for cross-validation.

    Args:
    n_splits (int): Number of splits.
    df (pyspark.sql.DataFrame): Input DataFrame.

    Returns:
    tuple: (blocks, df_with_row_num)
        - blocks: List of [start, mid, stop] indices for each split.
        - df_with_row_num: DataFrame with added row_num column.
    """
    
    
    w = Window.orderBy("index")
    df_with_row_num = df.withColumn("row_num", F.row_number().over(w))

    n_samples = df.count()
    k_fold_size = n_samples // n_splits

    blocks = []
    for i in range(n_splits):
        start = i * k_fold_size + 1  # +1 because row_number starts at 1
        stop = min(start + k_fold_size - 1, n_samples)  # Ensure we don't exceed n_samples
        mid = int(0.8 * (stop - start)) + start

        block = [start, mid, stop]
        blocks.append(block)
    # Add a row number column to the df and return the list of indexes and the new df
    return blocks, df_with_row_num


def evaluate_roc_curve(predictions, target_feature):
    evaluator = BinaryClassificationEvaluator(labelCol=target_feature)
    area_under_curve = evaluator.evaluate(predictions)
    return area_under_curve


def evaluate_multiclass_metrics(predictions, target_feature):
    # Convert DataFrame to RDD
    prediction_and_label = predictions\
                            .select(["prediction", target_feature])\
                            .withColumn(target_feature, col(target_feature).cast(FloatType()))\
                            .orderBy(target_feature)

    # Create MulticlassMetrics object
    metrics = MulticlassMetrics(prediction_and_label.rdd.map(tuple))

    # Collect metrics
    accuracy = metrics.accuracy
    precision = metrics.precision(1.0)
    recall = metrics.recall(1.0)
    f1_measure = metrics.fMeasure(1.0)
    
    return accuracy, precision, recall, f1_measure

In [0]:
# Cross Validation XG Boost for training
def xgboost_cv(blocks, df, max_depth=3, subsample=0.5, colsample_bytree=0.5):

    target_feature = "target"
    acc, prec, rec, f1, fb, res_frame = [], [], [], [], [], []

    error_flag = []


    xgb_regressor = SparkXGBClassifier(
        features_col="features",
        label_col=target_feature,
        num_workers=2,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree
        )


    for block in blocks:
        start_index, end_index, end_val = block

        train_block = df.filter((col("row_num") >= start_index) & (col("row_num") < end_index))
        val_block = df.filter((col("row_num") >= end_index) & (col("row_num") < end_val))

        xgb_model = xgb_regressor.fit(train_block)

        val_preds = xgb_model.transform(val_block)

        

        try:
            res_frame.append(val_preds)#.select(['QUARTER','FL_DATE','CRS_DEP_TIME','DEP_DEL15', 'prediction']))

            accuracy, precision, recall, f1_measure = evaluate_multiclass_metrics(val_preds, target_feature)
            beta = 1/(2**0.5)
            fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
        
            acc.append(accuracy)
            prec.append(precision)
            rec.append(recall)
            f1.append(f1_measure)
            fb.append(fbeta)
            error_flag.append(0)
        except:
            res_frame.append(val_preds)
            error_flag.append(1)
            res_frame.append(-1)

    def safe_mean(l):
        if len(l) == 0:
            return -1 
        else:
            return np.mean(l)
            
    avg_acc = safe_mean(acc)
    avg_prec = safe_mean(prec)
    avg_rec = safe_mean(rec)
    avg_f1 = safe_mean(f1)
    avg_fb = safe_mean(fb)

    print("\nParams for the Trial:")
    print(f"Max_depth: {max_depth}, Subsample: {subsample}, Colsample_bytree: {colsample_bytree}")

    print("\nAverage Metrics Across All Blocks:")
    print(f"Accuracy: {avg_acc:.4f}")
    print(f"Precision: {avg_prec:.4f}")
    print(f"Recall: {avg_rec:.4f}")
    print(f"F-Beta: {avg_fb:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")
    #print(f"AUC: {avg_auc:.4f}")

    return {
        'max_depth': max_depth,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'avg_accuracy': float(avg_acc),
        'avg_precision': float(avg_prec),
        'avg_recall': float(avg_rec),
        'avg_f1': float(avg_f1),
        'avg_f_beta': float(avg_fb)
    }

In [0]:
# Single Data Pass Through XG Boost for training the best model
# And evaluating the test dataset
def xgboost_model(train_block, test_block, max_depth=3, subsample=0.5, colsample_bytree=0.5):

    target_feature = "target"

    xgb_regressor = SparkXGBClassifier(
        features_col="features",
        label_col=target_feature,
        num_workers=2,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree
        )

    xgb_model = xgb_regressor.fit(train_block)

    test_preds = xgb_model.transform(test_block)   

    try:
        res_frame = test_preds

        accuracy, precision, recall, f1_measure = evaluate_multiclass_metrics(test_preds, target_feature)
        beta = 1/(2**0.5)
        fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
        error_flag = 0
    except:
        res_frame = test_preds
        error_flag = 1
        res_frame = -1

    def safe_value(val):
        if val is None:
            return -1 
        else:
            return val
        
    accuracy = safe_value(accuracy)
    recall = safe_value(recall)
    precision = safe_value(precision)
    f1_measure = safe_value(f1_measure)
    fbeta = safe_value(fbeta)     

    print("\nParams for the Trial:")
    print(f"Max_depth: {max_depth}, Subsample: {subsample}, Colsample_bytree: {colsample_bytree}")

    print("\nAverage Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F-Beta: {fbeta:.4f}")
    print(f"F1 Score: {f1_measure:.4f}")
    #print(f"AUC: {avg_auc:.4f}")

    return {
        'max_depth': max_depth,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'avg_accuracy': float(accuracy),
        'avg_precision': float(precision),
        'avg_recall': float(recall),
        'avg_f1': float(f1_measure),
        'avg_f_beta': float(fbeta)
    }

## Cross Validation Training

In [0]:
# Set up the Grid Search
from itertools import product

blocks, val_df_with_row_num = block_splits(3, val_trf)
cv_results = []
param_grid = {
    'max_depth':[3,6,10],
    'subsample':[0.5,0.75,1.0],
    'colsample_bytree':[0.5,0.75,1.0]
}
keys, values = zip(*param_grid.items())
all_combinations = [dict(zip(keys, combination)) for combination in product(*values)]

In [0]:
# Main training loop  - Performs Grid Search 
#   - - Trains multiple models = len(all_combinations) 
# WARNING!!! ------> THIS TAKES OVER AN HOUR
for params in all_combinations:
    print(f"current params: {params}")
    cv_results.append(xgboost_cv(blocks, val_df_with_row_num, **params))
schema = StructType([
    StructField('max_depth', IntegerType(), True),
    StructField('subsample', FloatType(), True),
    StructField('colsample_bytree', FloatType(), True),
    StructField('avg_accuracy', FloatType(), True),
    StructField('avg_precision', FloatType(), True),
    StructField('avg_recall', FloatType(), True),
    StructField('avg_f1', FloatType(), True),
    StructField('avg_f_beta', FloatType(), True),
])

cv_results_df = spark.createDataFrame(cv_results, schema=schema)

# Print the best result after saving the df of all results
# Now you know your best hyperparameters
best_result = max(cv_results, key=lambda x: x['avg_f_beta'])
print(f"Best result: {best_result}")
print(f"Highest avg_f_beta: {best_result['avg_f_beta']}")

current params: {'max_depth': 3, 'subsample': 0.5, 'colsample_bytree': 0.5}


2024-08-07 15:28:18,518 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.5, 'device': 'cpu', 'max_depth': 3, 'subsample': 0.5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 15:28:40,627 INFO XGBoost-PySpark: _fit Finished xgboost training!
/databricks/spark/python/pyspark/sql/context.py:164: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
2024-08-07 15:29:48,631 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.5, 'device': 'cpu', 'max_depth': 3, 'subsample': 0.5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 15:30:04,193 INFO XGBoost-PySpark: _fit Finished xg


Params for the Trial:
Max_depth: 3, Subsample: 0.5, Colsample_bytree: 0.5

Average Metrics Across All Blocks:
Accuracy: 0.7482
Precision: 0.7441
Recall: 0.5461
F-Beta: 0.6630
F1 Score: 0.6291
current params: {'max_depth': 3, 'subsample': 0.5, 'colsample_bytree': 0.75}


2024-08-07 15:31:39,084 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.75, 'device': 'cpu', 'max_depth': 3, 'subsample': 0.5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 15:31:49,233 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-08-07 15:32:24,144 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.75, 'device': 'cpu', 'max_depth': 3, 'subsample': 0.5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 15:32:33,446 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-08-07 15:33:40,219 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logist


Params for the Trial:
Max_depth: 3, Subsample: 0.5, Colsample_bytree: 0.75

Average Metrics Across All Blocks:
Accuracy: 0.7484
Precision: 0.7441
Recall: 0.5469
F-Beta: 0.6633
F1 Score: 0.6296
current params: {'max_depth': 3, 'subsample': 0.5, 'colsample_bytree': 1.0}


...


Params for the Trial:
Max_depth: 10, Subsample: 1.0, Colsample_bytree: 0.5

Average Metrics Across All Blocks:
Accuracy: 0.7452
Precision: 0.7363
Recall: 0.5466
F-Beta: 0.6590
F1 Score: 0.6265
current params: {'max_depth': 10, 'subsample': 1.0, 'colsample_bytree': 0.75}


2024-08-07 16:39:06,504 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.75, 'device': 'cpu', 'max_depth': 10, 'subsample': 1.0, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 16:39:20,400 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-08-07 16:40:03,006 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.75, 'device': 'cpu', 'max_depth': 10, 'subsample': 1.0, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 16:40:15,527 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-08-07 16:40:56,406 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logi


Params for the Trial:
Max_depth: 10, Subsample: 1.0, Colsample_bytree: 0.75

Average Metrics Across All Blocks:
Accuracy: 0.7456
Precision: 0.7374
Recall: 0.5461
F-Beta: 0.6594
F1 Score: 0.6267
current params: {'max_depth': 10, 'subsample': 1.0, 'colsample_bytree': 1.0}


2024-08-07 16:41:51,696 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 1.0, 'device': 'cpu', 'max_depth': 10, 'subsample': 1.0, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 16:42:03,771 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-08-07 16:42:45,058 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 1.0, 'device': 'cpu', 'max_depth': 10, 'subsample': 1.0, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 16:42:57,725 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-08-07 16:43:44,675 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logist


Params for the Trial:
Max_depth: 10, Subsample: 1.0, Colsample_bytree: 1.0

Average Metrics Across All Blocks:
Accuracy: 0.7454
Precision: 0.7370
Recall: 0.5459
F-Beta: 0.6590
F1 Score: 0.6263
Best result: {'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 1.0, 'avg_accuracy': 0.7484984860080525, 'avg_precision': 0.7439035905250303, 'avg_recall': 0.5474377881338233, 'avg_f1': 0.6299505464410227, 'avg_f_beta': 0.6636078207142596}
Highest avg_f_beta: 0.6636078207142596


# Final Train and Test

### Test Data Merge

In [0]:
# Similar to the train merge, we start with the NN files as those are already scoped to the year we are interested in - 2019
nntest2 = nntest2.withColumnRenamed("probability", "nnval2_prob") # keeping the col name val so I can recycle my pipeline
merged_nn = nntest2.join(nntest5, "index", "left") \
    .select("index", "nnval2_prob", F.col("probability").alias("nnval5_prob")) # keeping the col name val so I can recycle my pipeline 
	
merged_log_base = merged_nn.join(log_base, "index", "left") \
    .select("index", "nnval2_prob", "nnval5_prob", 
            F.col("DEP_DEL15").alias("target"), F.col("probability_1").alias("log_base_prob"), "YEAR")
			
merged_log_all = merged_log_base.join(log_variant, "index", "left") \
    .select("index", "nnval2_prob", "nnval5_prob", "log_base_prob",
            F.col("probability_1").alias("log_variant_prob"), "target")
			
merged_boost11 = merged_log_all.join(xgboost1011, "index", "left")\
    .select("index", "nnval2_prob", "nnval5_prob", "log_base_prob", "log_variant_prob",
            F.col("probability_1").alias("xgboost1011_prob"), "target")
			
test_df = merged_boost11.join(xgboost1066, "index", "left")\
    .select("index", "nnval2_prob", "nnval5_prob", "log_base_prob", "log_variant_prob", "xgboost1011_prob",
            F.col("probability_1").alias("xgboost1066_prob"), "target")

# Count total number of rows
total_rows = test_df.count()
print(f"Total number of rows: {total_rows}")

# Check for null data in any column
null_counts = test_df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in test_df.columns])
null_counts.show()

# Count rows with any null value
rows_with_nulls = test_df.filter(F.concat_ws("", *test_df.columns).isNull()).count()
print(f"Number of rows with null values: {rows_with_nulls}")

Total number of rows: 7270189
+-----+-----------+-----------+-------------+----------------+----------------+----------------+------+
|index|nnval2_prob|nnval5_prob|log_base_prob|log_variant_prob|xgboost1011_prob|xgboost1066_prob|target|
+-----+-----------+-----------+-------------+----------------+----------------+----------------+------+
|    0|          0|          0|            0|         6205149|               0|               0|     0|
+-----+-----------+-----------+-------------+----------------+----------------+----------------+------+

Number of rows with null values: 0


#### Transform

In [0]:
# Use the same pipeline from training on the train data
test_trf = pipeline \
    .fit(val_df) \
    .transform(test_df) \
    .select("index", "features", "target")

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

#### Train and Predict

In [0]:
test_results = xgboost_model(val_trf, test_trf, max_depth=3, subsample=1, colsample_bytree=1)

2024-08-07 16:46:20,335 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 1, 'device': 'cpu', 'max_depth': 3, 'subsample': 1, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-07 16:46:40,010 INFO XGBoost-PySpark: _fit Finished xgboost training!
/databricks/spark/python/pyspark/sql/context.py:164: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(



Params for the Trial:
Max_depth: 3, Subsample: 1, Colsample_bytree: 1

Average Metrics:
Accuracy: 0.7549
Precision: 0.5260
Recall: 0.6632
F-Beta: 0.5650
F1 Score: 0.5867
